In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
df_train = reduce_memory_usage(df_train)
df_train = df_train.set_index('id')
df_train.head()

In [ ]:
df_train.target.unique()

In [ ]:
df_train.target.value_counts()

In [ ]:
df_train.target.value_counts()/len(df_train)

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
df_test = reduce_memory_usage(df_test)
df_test = df_test.set_index('id')
df_test.head()

In [ ]:
X = df_train.drop('target', axis = 1)
X.head()

In [ ]:
y = df_train.target
y

In [ ]:
X_enc = X.copy()
X_enc

In [ ]:
X_enc

In [ ]:
cat_cols = list(X_enc.select_dtypes('object', 'category').columns)
cat_cols

In [ ]:
X_enc.f_27.value_counts()

In [ ]:
X_enc.f_27

In [ ]:
for i in range(10):
    X_enc['f_27' + '_' + str(i)] = X_enc.f_27.str.get(i).apply(ord) - ord('A')

In [ ]:
X_enc

In [ ]:
import category_encoders as ce
encoder = ce.OrdinalEncoder()
encoder.fit(X_enc['f_27'])
X_enc['f_27'] = encoder.transform(X_enc['f_27'])
X_enc['f_27']

In [ ]:
test_enc = df_test.copy()
test_enc

In [ ]:
for i in range(10):
    test_enc['f_27' + '_' + str(i)] = test_enc.f_27.str.get(i).apply(ord) - ord('A')

In [ ]:
encoder = ce.OrdinalEncoder()
encoder.fit(test_enc['f_27'])
test_enc['f_27'] = encoder.transform(test_enc['f_27'])
test_enc['f_27']

In [ ]:
test_enc

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=101, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
scaler.fit(test_enc)
test_enc = scaler.transform(test_enc)

In [ ]:
test_enc

In [ ]:
X_train.shape

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [ ]:
test_enc = np.asarray(test_enc).astype(np.float32)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.BatchNormalization(input_shape=[41]),
    layers.Dense(1024, activation='relu', input_shape=[41]),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1024, activation='relu'),    
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy', 'AUC'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)


history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=512,
    epochs=100,
    callbacks=[early_stopping],
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()
#history_df.loc[5:, ['AUC', 'val_AUC']].plot()


print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

In [ ]:
preds = model.predict(X_test)
preds

In [ ]:
from sklearn.metrics import roc_auc_score
nn_roc = roc_auc_score(y_test, preds)
nn_roc

In [ ]:
preds = model.predict(test_enc)
preds

In [ ]:
df_subb = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')
df_subb

In [ ]:
df_subb.target = preds
df_subb

In [ ]:
df_subb.to_csv('subb.csv', index = None)